# Bicycle sales analysis

## Units sold by Categories, Brands and products

First we are going to select categories, brands, products and stocks.

In [68]:
SELECT TOP 3 *
--- We subquery to get the total sales grouped by category_name from 3 different tables applying the join method.
FROM (
    SELECT
        c.category_name, --- Category name from production.categories table
        SUM(s.quantity) as units_sold --- This calculates the total sales from the sales.order_items s main table
    FROM 
        sales.order_items s --- Main table
    JOIN 
        production.products p ON p.product_id = s.product_id --- Join of production.products table
    JOIN 
        production.categories c ON p.category_id = c.category_id --- Join of production.categories table
    GROUP BY  
        c.category_name -- Grouped by category_name
) AS sub --- Alias
ORDER BY 
    units_sold DESC;

As we can see, Cruisers Bicycles, Mountain Bikes and Children Bicycles are the most sold bikes categories.

In [70]:
SELECT TOP 3 *
--- We subquery to get the total sales grouped by category_name from 3 different tables applying the join method.
FROM (
    SELECT
        p.product_name, --- Category name from production.categories table
        SUM(s.quantity) as units_sold --- This calculates the total sales from the sales.order_items s main table
    FROM 
        sales.order_items s --- Main table
    JOIN 
        production.products p ON p.product_id = s.product_id --- Join of production.products table

    GROUP BY  
        p.product_name -- Grouped by product_name
) AS sub --- Alias
ORDER BY 
    units_sold DESC;

,product_name,units_sold
0,Electra Cruiser 1 (24-Inch) - 2016,296
1,Electra Townie Original 7D EQ - 2016,290
2,Electra Townie Original 21D - 2016,289


In [72]:
SELECT TOP 3 *
--- We subquery to get the total sales grouped by category_name from 3 different tables applying the join method.
FROM (
    SELECT
        b.brand_name, --- Category name from production.categories table
        SUM(s.quantity) as units_sold --- This calculates the total sales from the sales.order_items s main table
    FROM 
        sales.order_items s --- Main table
    JOIN 
        production.products p ON p.product_id = s.product_id --- Join of production.products table
	JOIN 
        production.brands b ON b.brand_id = p.brand_id --- Join of production.brands table
    GROUP BY  
        b.brand_name -- Grouped by brand_name
) AS sub --- Alias
ORDER BY 
    units_sold DESC;

,brand_name,units_sold
0,Electra,2612
1,Trek,1839
2,Surly,908


As we can see from the above results, top 3 categories, brands and products are:

Categories
1. Cruisers Bicycles / 2,063 units sold
2. Mountain Bikes / 1,755 units sold
3. Children Bicycles / 1,179 units sold

Products
1. Electra Cruiser 1 (24-Inch) - 2016 / 296 units sold
2. Electra Townie Original 7D EQ - 2016 / 290 units sold
3. Electra Townie Original 21D - 2016 289 / units sold


Brands
1. Electra - 2,612 units sold
2. Trek - 1,839 units sold
3. Surly - 908 units sold

Electra sales represents the most important brand for the company, with more than 2.5k units sold last year, just above Trek and Surly. Most sold categories are Cruisers Bicycles, Mountain Bikes and Children Bicycles. Finally, Electra leads the top 3 most sold products.

## Units sold by store and product

In [9]:
SELECT * FROM (
SELECT ss.store_name as store, --- store name
       pp.product_name as product, --- product name
	   SUM(s.quantity) as total_units,--- total units sold
       ROW_NUMBER() OVER (PARTITION BY ss.store_name ORDER BY SUM(s.quantity) DESC) as rank --- ranking
FROM sales.order_items s
JOIN sales.orders so ON so.order_id = s.order_id --- join orders table and brings store id
JOIN sales.stores ss ON so.store_id = ss.store_id --- join stores table and brings store name
JOIN production.products pp ON pp.product_id = s.product_id --- join products table and brings product name
GROUP BY  ss.store_name,pp.product_name
	) as sub
WHERE rank IN (1,2,3)

Now, in the aggregation by store and product, just as expected, Electra leads sales in all of the three stores.

Baldwin Bikes:

- Electra Cruiser 1 (24-Inch) - 2016 / 211 units sold
- Electra Townie Original 7D EQ - 2016 / 205 units sold
- Electra Townie Original 21D - 2016 / 203 units sold

Santa Cruz Bikes:
- Electra Girl's Hawaii 1 (16-inch) - 2015/2016 / 59 units sold
- Electra Townie Original 7D EQ - 2016 / 56 units sold
- Electra Townie Original 21D - 2016 / 52 units sold

Rowlett Bikes: 
- Electra Cruiser 1 (24-Inch) - 2016 / 41 units sold
- Electra Townie Original 21D - 2016 / 34 units sold
- Electra Girl's Hawaii 1 (16-inch) - 2015/2016 / 29 units sold

## Total orders by Year and Month

In [21]:
--- This is the year and month sales by total in desc order

SELECT DATENAME(yy,so.order_date) as date_year, --- this subtract the year
	   DATENAME(mm,so.order_date) as date_month, --- this subtract the  month
       COUNT(so.order_id) as total_orders --- this get the total
       FROM sales.orders so
GROUP BY DATENAME(yy,so.order_date),DATENAME(mm,so.order_date) --- grouped by year the month
	ORDER BY total_orders DESC
	

,date_year,date_month,total_orders
0,2018,April,125
1,2018,March,68
2,2017,March,67
3,2016,September,67
4,2017,October,65
5,2017,August,65
6,2016,October,64
7,2016,August,63
8,2017,June,63
9,2017,May,57


As we can see, top sales was perfomed in april 2018.

## Most loyal clients

In [7]:
--- This is the purchases by customer by year and month
SELECT * FROM  (
SELECT DATEPART(yy,so.order_date) as date_year, --- this subtract the year
	   DATEPART(mm,so.order_date) as date_month, --- this subtract the month
	   sc.first_name as name,
       COUNT(so.order_id) as total_orders, --- this get the total
	   ROW_NUMBER() OVER (PARTITION BY sc.first_name ORDER BY COUNT(so.order_id)) as purchase_frecuency --- this count the times the customer name appears meaning the number of purchases.
       FROM sales.orders so
JOIN sales.customers sc ON sc.customer_id = so.customer_id
GROUP BY DATEPART(yy,so.order_date),DATEPART(mm,so.order_date),sc.first_name
	) as sales_date
	WHERE purchase_frecuency >= 3 --- This filters only customers who have done 3 or more purchases
	ORDER BY  name , purchase_frecuency DESC

,date_year,date_month,name,total_orders,purchase_frecuency
0,2016,1,Alane,1,3
1,2018,4,Aleta,1,5
2,2016,8,Aleta,1,4
3,2016,11,Aleta,1,3
4,2018,4,Araceli,1,3
...,...,...,...,...,...
79,2018,7,Thalia,1,3
80,2017,10,Tobie,1,3
81,2018,4,Tommie,1,3
82,2018,4,Williemae,1,3


In [5]:
SELECT 
    SUM(CASE WHEN DATENAME(yy, so.order_date) = '2016' THEN 1 ELSE 0 END) as [2016],
    SUM(CASE WHEN DATENAME(yy, so.order_date) = '2017' THEN 1 ELSE 0 END) as [2017],
	SUM(CASE WHEN DATENAME(yy, so.order_date) = '2018' THEN 1 ELSE 0 END) as [2018],
    sc.first_name as name
FROM sales.orders so
JOIN sales.customers sc ON sc.customer_id = so.customer_id
GROUP BY sc.first_name
HAVING SUM(CASE WHEN DATENAME(yy, so.order_date) = '2016' THEN 1 ELSE 0 END) = 1 
      AND SUM(CASE WHEN DATENAME(yy, so.order_date) = '2017' THEN 1 ELSE 0 END) = 1
	    AND SUM(CASE WHEN DATENAME(yy, so.order_date) = '2018' THEN 1 ELSE 0 END) = 1
ORDER BY name ASC



,2016,2017,2018,name
0,1,1,1,Bobbie
1,1,1,1,Brittney
2,1,1,1,Cesar
3,1,1,1,Emmitt
4,1,1,1,Georgeann
5,1,1,1,Jacquline
6,1,1,1,Jeromy
7,1,1,1,Kanesha
8,1,1,1,Lashawn
9,1,1,1,Merlene


In the first table a column with the purchase frecuency is shown. The more the frecuency is the more loyal the client is. We can now use this table to show only people who have purchased in the current year, last year, and so on. 
As an alternative (Depending on the focusing and perspective), the second table show customers who have made a purchase in each of the analized years, where 1 is for existing data, and 0 is for not existing data.

## Total orders by Staff member

In [22]:
SELECT so.staff_id as id, --- This gets the id
       COUNT(so.order_id) as total_orders --- this get the total units purchased
       FROM sales.orders so
WHERE so.staff_id IN (
SELECT st.staff_id FROM sales.staffs as st WHERE st.active = 1) --- This filters only active employees where 1 means TRUE
GROUP BY so.staff_id
	ORDER BY total_orders desc
	

,id,total_orders
0,6,553
1,7,540
2,3,184
3,2,164
4,8,88
5,9,86


As for the above results, we can see that staff with id 6 has done the most sales.